# 📏 Feature Scaling y Optimización del Learning Rate

## 📚 Objetivos de Aprendizaje
En este notebook aprenderás:
- **Por qué** el feature scaling es crucial para el gradient descent
- **Cómo** implementar diferentes técnicas de normalización
- **Cuándo** y **cómo** ajustar el learning rate
- **Técnicas avanzadas** para acelerar la convergencia

## 🔍 Problema Identificado
En el notebook anterior observamos:
- **Learning rate muy pequeño** (5.0e-7) necesario para evitar divergencia
- **Convergencia lenta** debido a diferentes escalas de features
- **Gradientes desbalanceados** entre diferentes parámetros

**Solución**: Feature Scaling + Learning Rate Optimization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math

# Configuración
plt.style.use('default')
np.set_printoptions(precision=2, suppress=True)

print("✅ Librerías importadas")
print("🎯 Listo para optimizar gradient descent")

## 1. 📊 Análisis del Problema: Escalas Desbalanceadas

### 1.1 Cargar Dataset y Analizar Escalas

In [ ]:
# Cargar el mismo dataset de casas
def cargar_datos_casas():
    X_train = np.array([
        [2104, 5, 1, 45],    # Casa 1: [tamaño_sqft, habitaciones, pisos, edad]
        [1416, 3, 2, 40],    # Casa 2  
        [852,  2, 1, 35],    # Casa 3
        [1940, 4, 1, 10],    # Casa 4
        [1539, 3, 2, 25],    # Casa 5
        [3000, 4, 2, 8],     # Casa 6
        [1230, 3, 1, 15],    # Casa 7
        [2145, 4, 1, 12],    # Casa 8
        [1736, 3, 2, 30],    # Casa 9
        [1000, 2, 1, 50],    # Casa 10
        [1940, 4, 3, 5],     # Casa 11 (nueva)
        [1100, 2, 1, 60]     # Casa 12 (nueva)
    ])
    
    y_train = np.array([460, 232, 178, 500, 315, 740, 285, 510, 390, 180, 550, 165])
    feature_names = ['Tamaño (sqft)', 'Habitaciones', 'Pisos', 'Edad (años)']
    
    return X_train, y_train, feature_names

X_train, y_train, feature_names = cargar_datos_casas()
m, n = X_train.shape

print(f"📊 Dataset actualizado:")
print(f"   • Ejemplos (m): {m}")
print(f"   • Features (n): {n}")

# Análisis detallado de escalas
print(f"\n📏 Análisis de Escalas por Feature:")
print(f"{'Feature':<15} {'Min':<8} {'Max':<8} {'Media':<8} {'Std':<8} {'Rango':<10}")
print("-" * 65)

for i, feature in enumerate(feature_names):
    columna = X_train[:, i]
    min_val = np.min(columna)
    max_val = np.max(columna)
    media = np.mean(columna)
    std = np.std(columna)
    rango = max_val - min_val
    
    print(f"{feature:<15} {min_val:<8.0f} {max_val:<8.0f} {media:<8.1f} {std:<8.1f} {rango:<10.0f}")

print(f"\n🔍 Observaciones críticas:")
print(f"   • Tamaño: Rango ~2000 (852-3000)")
print(f"   • Habitaciones: Rango ~3 (2-5)")
print(f"   • Pisos: Rango ~2 (1-3)")
print(f"   • Edad: Rango ~55 (5-60)")
print(f"\n⚠️  PROBLEMA: Tamaño domina numéricamente sobre otras features")

### 1.2 Visualización del Problema de Escalas

In [ ]:
# Visualizar el problema de escalas
plt.figure(figsize=(15, 10))

# Subplot 1: Box plots para mostrar diferencias de escala
plt.subplot(2, 3, 1)
box_data = [X_train[:, i] for i in range(n)]
plt.boxplot(box_data, labels=[name.split()[0] for name in feature_names])
plt.title('Distribución de Features (Escala Original)')
plt.ylabel('Valores')
plt.yscale('log')  # Escala logarítmica para ver todas las features
plt.grid(True, alpha=0.3)

# Subplot 2: Histogramas individuales
for i in range(n):
    plt.subplot(2, 3, i+2)
    plt.hist(X_train[:, i], bins=8, alpha=0.7, edgecolor='black')
    plt.title(f'{feature_names[i]}')
    plt.xlabel('Valor')
    plt.ylabel('Frecuencia')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Demonstrar el impacto en los gradientes
def calcular_gradientes_ejemplo(X, y, w, b):
    """Función simplificada para demostrar gradientes"""
    m, n = X.shape
    dj_dw = np.zeros(n)
    dj_db = 0.
    
    for i in range(m):
        error = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] += error * X[i, j]
        dj_db += error
    
    return dj_dw / m, dj_db / m

# Calcular gradientes iniciales
w_test = np.array([0.1, 10, -20, -1])  # Valores ejemplo
b_test = 100

gradientes, grad_b = calcular_gradientes_ejemplo(X_train, y_train, w_test, b_test)

print(f"\n🧮 Impacto en Gradientes (sin normalización):")
print(f"{'Feature':<15} {'Gradiente':<12} {'Magnitud':<10}")
print("-" * 40)
for i, (feature, grad) in enumerate(zip(feature_names, gradientes)):
    print(f"{feature:<15} {grad:<12.2e} {abs(grad):<10.2e}")

print(f"\n📊 Ratio de magnitudes:")
grad_abs = np.abs(gradientes)
ratio_max_min = np.max(grad_abs) / np.min(grad_abs[grad_abs > 0])
print(f"   Máximo/Mínimo: {ratio_max_min:.1f}x de diferencia")
print(f"   Esto causa convergencia desigual y requiere learning rate muy pequeño")

## 2. 🎯 Técnicas de Feature Scaling

### 2.1 Z-Score Normalization (Standardization)

**Fórmula**: $x_{norm} = \frac{x - \mu}{\sigma}$

**Resultado**: Media = 0, Desviación estándar = 1

In [ ]:
# Implementación de Z-Score Normalization

def zscore_normalize_features(X):
    """
    Aplica normalización Z-score a las features
    
    Args:
        X (ndarray): Matriz de features (m, n)
        
    Returns:
        X_norm (ndarray): Features normalizadas
        mu (ndarray): Media de cada feature
        sigma (ndarray): Desviación estándar de cada feature
    """
    # Calcular estadísticas
    mu = np.mean(X, axis=0)      # Media por columna
    sigma = np.std(X, axis=0)    # Desviación estándar por columna
    
    # Normalizar
    X_norm = (X - mu) / sigma
    
    return X_norm, mu, sigma

# Aplicar Z-score normalization
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)

print(f"📊 Estadísticas antes de normalización:")
print(f"   Media por feature: {X_mu}")
print(f"   Std por feature:   {X_sigma}")

print(f"\n📊 Estadísticas después de normalización:")
print(f"   Media por feature: {np.mean(X_norm, axis=0)}")
print(f"   Std por feature:   {np.std(X_norm, axis=0)}")

print(f"\n📏 Comparación de rangos:")
print(f"{'Feature':<15} {'Rango Original':<15} {'Rango Normalizado':<18}")
print("-" * 50)

for i, feature in enumerate(feature_names):
    rango_original = np.ptp(X_train[:, i])  # peak-to-peak
    rango_norm = np.ptp(X_norm[:, i])
    print(f"{feature:<15} {rango_original:<15.0f} {rango_norm:<18.2f}")

print(f"\n✅ Normalización exitosa: Todas las features ahora tienen escala similar")

### 2.2 Min-Max Normalization

**Fórmula**: $x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}$

**Resultado**: Valores entre 0 y 1

In [ ]:
# Implementación de Min-Max Normalization

def minmax_normalize_features(X):
    """
    Aplica normalización Min-Max a las features
    
    Args:
        X (ndarray): Matriz de features (m, n)
        
    Returns:
        X_norm (ndarray): Features normalizadas [0, 1]
        X_min (ndarray): Valor mínimo de cada feature
        X_max (ndarray): Valor máximo de cada feature
    """
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)
    
    # Evitar división por cero
    X_range = X_max - X_min
    X_range[X_range == 0] = 1  # Si max == min, mantener valor original
    
    X_norm = (X - X_min) / X_range
    
    return X_norm, X_min, X_max

# Aplicar Min-Max normalization
X_minmax, X_min, X_max = minmax_normalize_features(X_train)

print(f"📊 Min-Max Normalization:")
print(f"   Valores mínimos: {X_min}")
print(f"   Valores máximos: {X_max}")

print(f"\n📊 Estadísticas después de Min-Max:")
print(f"   Min por feature: {np.min(X_minmax, axis=0)}")
print(f"   Max por feature: {np.max(X_minmax, axis=0)}")
print(f"   Media por feature: {np.mean(X_minmax, axis=0)}")

# Comparar ambas técnicas visualmente
plt.figure(figsize=(15, 5))

# Original
plt.subplot(1, 3, 1)
for i in range(n):
    plt.scatter([i] * len(X_train), X_train[:, i], alpha=0.6, s=50)
plt.title('Features Originales')
plt.xlabel('Feature Index')
plt.ylabel('Valor')
plt.yscale('log')
plt.xticks(range(n), [f.split()[0] for f in feature_names])
plt.grid(True, alpha=0.3)

# Z-Score
plt.subplot(1, 3, 2)
for i in range(n):
    plt.scatter([i] * len(X_norm), X_norm[:, i], alpha=0.6, s=50)
plt.title('Z-Score Normalization')
plt.xlabel('Feature Index')
plt.ylabel('Valor Normalizado')
plt.xticks(range(n), [f.split()[0] for f in feature_names])
plt.grid(True, alpha=0.3)

# Min-Max
plt.subplot(1, 3, 3)
for i in range(n):
    plt.scatter([i] * len(X_minmax), X_minmax[:, i], alpha=0.6, s=50)
plt.title('Min-Max Normalization')
plt.xlabel('Feature Index')
plt.ylabel('Valor Normalizado [0,1]')
plt.xticks(range(n), [f.split()[0] for f in feature_names])
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🤔 ¿Cuál usar?")
print(f"   • Z-Score: Mejor para distribuciones normales, no acotado")
print(f"   • Min-Max: Mejor cuando necesitas rango específico [0,1]")
print(f"   • Para ML: Z-Score es más común y robusto")

## 3. ⚡ Impacto en Gradient Descent

### 3.1 Comparación: Con vs Sin Feature Scaling

In [ ]:
# Funciones auxiliares (copiadas del notebook anterior)
def calcular_costo_multivariable(X, y, w, b):
    m = X.shape[0]
    costo = 0.0
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b
        costo += (f_wb_i - y[i]) ** 2
    return costo / (2 * m)

def calcular_gradientes_multivariable(X, y, w, b):
    m, n = X.shape
    dj_dw = np.zeros(n)
    dj_db = 0.
    
    for i in range(m):
        error = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] += error * X[i, j]
        dj_db += error
    
    return dj_dw / m, dj_db / m

def gradient_descent_mejorado(X, y, w_inicial, b_inicial, alpha, num_iteraciones, verbose=True):
    """Gradient descent con tracking mejorado"""
    w = copy.deepcopy(w_inicial)
    b = b_inicial
    historial_costos = []
    historial_params = []
    
    for i in range(num_iteraciones):
        # Calcular costo y gradientes
        costo = calcular_costo_multivariable(X, y, w, b)
        dj_dw, dj_db = calcular_gradientes_multivariable(X, y, w, b)
        
        # Actualizar parámetros
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        # Guardar historia
        historial_costos.append(costo)
        historial_params.append((w.copy(), b))
        
        # Imprimir progreso
        if verbose and (i % (num_iteraciones // 10) == 0 or i < 10):
            print(f"Iter {i:4d}: Costo = {costo:8.4f}, |dj_dw| = {np.linalg.norm(dj_dw):.2e}")
    
    return w, b, historial_costos, historial_params

# Experimento: Comparar convergencia con y sin normalización
print(f"🧪 EXPERIMENTO: Convergencia Con vs Sin Feature Scaling")
print(f"=" * 60)

# Parámetros del experimento
w_inicial = np.zeros(n)
b_inicial = 0.
iteraciones = 1000

# Caso 1: Sin normalización (learning rate pequeño)
print(f"\n📉 Caso 1: SIN normalización (α = 5e-7)")
alpha_sin_norm = 5.0e-7
w1, b1, costos1, params1 = gradient_descent_mejorado(
    X_train, y_train, w_inicial, b_inicial, alpha_sin_norm, iteraciones, verbose=False
)

print(f"   Costo inicial: {costos1[0]:.2f}")
print(f"   Costo final:   {costos1[-1]:.2f}")
print(f"   Reducción:     {((costos1[0] - costos1[-1])/costos1[0]*100):.1f}%")

# Caso 2: Con normalización (learning rate grande)
print(f"\n📈 Caso 2: CON normalización (α = 0.1)")
alpha_con_norm = 0.1
w2, b2, costos2, params2 = gradient_descent_mejorado(
    X_norm, y_train, w_inicial, b_inicial, alpha_con_norm, iteraciones, verbose=False
)

print(f"   Costo inicial: {costos2[0]:.2f}")
print(f"   Costo final:   {costos2[-1]:.2f}")
print(f"   Reducción:     {((costos2[0] - costos2[-1])/costos2[0]*100):.1f}%")

# Comparación de velocidades
mejora_velocidad = alpha_con_norm / alpha_sin_norm
print(f"\n⚡ COMPARACIÓN:")
print(f"   Learning rate con normalización: {mejora_velocidad:.0f}x más grande")
print(f"   Convergencia: Mucho más rápida y estable")
print(f"   Costo final similar: {abs(costos1[-1] - costos2[-1]):.2f} diferencia")

### 3.2 Visualización de la Convergencia

In [ ]:
# Visualizar comparación de convergencia
plt.figure(figsize=(15, 10))

# Subplot 1: Evolución del costo (escala lineal)
plt.subplot(2, 3, 1)
plt.plot(costos1, 'r-', linewidth=2, label='Sin normalización', alpha=0.8)
plt.plot(costos2, 'b-', linewidth=2, label='Con normalización', alpha=0.8)
plt.title('Evolución del Costo (Lineal)')
plt.xlabel('Iteraciones')
plt.ylabel('Costo')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Evolución del costo (escala log)
plt.subplot(2, 3, 2)
plt.plot(costos1, 'r-', linewidth=2, label='Sin normalización', alpha=0.8)
plt.plot(costos2, 'b-', linewidth=2, label='Con normalización', alpha=0.8)
plt.title('Evolución del Costo (Log)')
plt.xlabel('Iteraciones')
plt.ylabel('Costo (log scale)')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Convergencia de parámetros w (sin normalización)
plt.subplot(2, 3, 3)
w_history1 = np.array([params[0] for params in params1])
for j in range(n):
    plt.plot(w_history1[:, j], label=f'w[{j}] ({feature_names[j].split()[0]})', linewidth=2)
plt.title('Parámetros w (Sin Normalización)')
plt.xlabel('Iteraciones')
plt.ylabel('Valor de w')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: Convergencia de parámetros w (con normalización)
plt.subplot(2, 3, 4)
w_history2 = np.array([params[0] for params in params2])
for j in range(n):
    plt.plot(w_history2[:, j], label=f'w[{j}] ({feature_names[j].split()[0]})', linewidth=2)
plt.title('Parámetros w (Con Normalización)')
plt.xlabel('Iteraciones')
plt.ylabel('Valor de w')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 5: Comparación de gradientes (magnitud)
plt.subplot(2, 3, 5)
# Calcular norma de gradientes en cada iteración
normas_grad1 = []
normas_grad2 = []

for i in range(0, len(costos1), 50):  # Cada 50 iteraciones para velocidad
    w_i, b_i = params1[i]
    dj_dw, _ = calcular_gradientes_multivariable(X_train, y_train, w_i, b_i)
    normas_grad1.append(np.linalg.norm(dj_dw))
    
    w_i, b_i = params2[i]
    dj_dw, _ = calcular_gradientes_multivariable(X_norm, y_train, w_i, b_i)
    normas_grad2.append(np.linalg.norm(dj_dw))

iters_sample = range(0, len(costos1), 50)
plt.plot(iters_sample, normas_grad1, 'r-', linewidth=2, label='Sin normalización', alpha=0.8)
plt.plot(iters_sample, normas_grad2, 'b-', linewidth=2, label='Con normalización', alpha=0.8)
plt.title('Magnitud de Gradientes')
plt.xlabel('Iteraciones')
plt.ylabel('||∇J|| (Norma del gradiente)')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 6: Superficie de costo (conceptual)
plt.subplot(2, 3, 6)
# Crear una representación 2D simplificada de la superficie de costo
w0_range = np.linspace(-2, 2, 50)
w1_range = np.linspace(-2, 2, 50)
W0, W1 = np.meshgrid(w0_range, w1_range)

# Función de costo simulada (elíptica para mostrar el concepto)
Z_normalized = (W0**2 + W1**2)  # Circular (normalizado)
Z_unnormalized = (W0**2 * 100 + W1**2)  # Elíptico (sin normalizar)

plt.contour(W0, W1, Z_normalized, levels=10, colors='blue', alpha=0.6, linewidths=1)
plt.contour(W0, W1, Z_unnormalized, levels=10, colors='red', alpha=0.6, linewidths=1)
plt.title('Superficie de Costo (Conceptual)')
plt.xlabel('w0')
plt.ylabel('w1')
plt.legend(['Normalizado (circular)', 'Sin normalizar (elíptico)'])
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 Análisis de las visualizaciones:")
print(f"   1. Costo: Ambos convergen, pero normalización es más rápida")
print(f"   2. Log scale: Muestra velocidad de convergencia superior")
print(f"   3. Parámetros sin norm: Convergencia desigual y oscilatoria")
print(f"   4. Parámetros con norm: Convergencia suave y uniforme")
print(f"   5. Gradientes: Decaen más rápido con normalización")
print(f"   6. Superficie: Forma circular vs elíptica explica la diferencia")

## 4. 🎛️ Optimización del Learning Rate

### 4.1 Búsqueda del Learning Rate Óptimo

In [ ]:
# Función para probar diferentes learning rates
def probar_learning_rates(X, y, learning_rates, iteraciones=500):
    """
    Prueba diferentes learning rates y retorna resultados
    """
    resultados = []
    
    for alpha in learning_rates:
        try:
            w, b, costos, _ = gradient_descent_mejorado(
                X, y, np.zeros(X.shape[1]), 0., alpha, iteraciones, verbose=False
            )
            
            # Verificar si divergió
            if np.isnan(costos[-1]) or np.isinf(costos[-1]) or costos[-1] > costos[0] * 10:
                status = "Diverge"
                costo_final = float('inf')
            else:
                status = "Converge"
                costo_final = costos[-1]
            
            resultados.append({
                'alpha': alpha,
                'costo_final': costo_final,
                'status': status,
                'costos': costos,
                'reduccion': ((costos[0] - costo_final) / costos[0] * 100) if costo_final != float('inf') else 0
            })
            
        except Exception as e:
            resultados.append({
                'alpha': alpha,
                'costo_final': float('inf'),
                'status': 'Error',
                'costos': [],
                'reduccion': 0
            })
    
    return resultados

# Probar rango amplio de learning rates
learning_rates_test = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0]

print(f"🧪 EXPERIMENTO: Búsqueda de Learning Rate Óptimo")
print(f"=" * 55)
print(f"Probando {len(learning_rates_test)} valores de α con datos normalizados...")

resultados_lr = probar_learning_rates(X_norm, y_train, learning_rates_test)

# Mostrar resultados
print(f"\n{'α (Learning Rate)':<15} {'Status':<10} {'Costo Final':<12} {'Reducción %':<12}")
print("-" * 50)

for resultado in resultados_lr:
    if resultado['costo_final'] == float('inf'):
        costo_str = "∞ (Diverge)"
        reduccion_str = "0.0%"
    else:
        costo_str = f"{resultado['costo_final']:.2f}"
        reduccion_str = f"{resultado['reduccion']:.1f}%"
    
    print(f"{resultado['alpha']:<15} {resultado['status']:<10} {costo_str:<12} {reduccion_str:<12}")

# Encontrar el mejor learning rate
resultados_validos = [r for r in resultados_lr if r['status'] == 'Converge']
if resultados_validos:
    mejor_resultado = min(resultados_validos, key=lambda x: x['costo_final'])
    print(f"\n🏆 Mejor learning rate: α = {mejor_resultado['alpha']}")
    print(f"   Costo final: {mejor_resultado['costo_final']:.4f}")
    print(f"   Reducción: {mejor_resultado['reduccion']:.1f}%")

### 4.2 Visualización de Learning Rates

In [ ]:
# Visualizar comportamiento de diferentes learning rates
plt.figure(figsize=(15, 10))

# Colores para cada learning rate
colores = plt.cm.viridis(np.linspace(0, 1, len(learning_rates_test)))

# Subplot 1: Evolución del costo para diferentes α
plt.subplot(2, 2, 1)
for i, (resultado, color) in enumerate(zip(resultados_lr, colores)):
    if resultado['costos'] and len(resultado['costos']) > 0:
        # Limitar costos para visualización
        costos_plot = np.array(resultado['costos'])
        costos_plot = np.clip(costos_plot, 0, 10000)  # Limitar valores extremos
        
        plt.plot(costos_plot, color=color, linewidth=2, 
                label=f"α={resultado['alpha']:.3f}" + 
                      (" (Diverge)" if resultado['status'] != 'Converge' else ""))

plt.title('Evolución del Costo vs Learning Rate')
plt.xlabel('Iteraciones')
plt.ylabel('Costo')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Subplot 2: Costo final vs learning rate
plt.subplot(2, 2, 2)
alphas_plot = []
costos_finales = []
colores_status = []

for resultado in resultados_lr:
    alphas_plot.append(resultado['alpha'])
    if resultado['costo_final'] == float('inf'):
        costos_finales.append(1000)  # Valor alto para visualización
        colores_status.append('red')
    else:
        costos_finales.append(resultado['costo_final'])
        colores_status.append('green')

plt.scatter(alphas_plot, costos_finales, c=colores_status, s=100, alpha=0.7)
plt.title('Costo Final vs Learning Rate')
plt.xlabel('Learning Rate (α)')
plt.ylabel('Costo Final')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, alpha=0.3)

# Añadir línea vertical en el mejor α
if resultados_validos:
    plt.axvline(x=mejor_resultado['alpha'], color='blue', linestyle='--', 
                linewidth=2, label=f'Mejor: α={mejor_resultado["alpha"]}')
    plt.legend()

# Subplot 3: Comparación específica de algunos α buenos
plt.subplot(2, 2, 3)
alphas_comparar = [0.01, 0.1, 1.0]  # Seleccionar algunos para comparación detallada

for alpha in alphas_comparar:
    # Encontrar resultado correspondiente
    resultado = next((r for r in resultados_lr if r['alpha'] == alpha), None)
    if resultado and resultado['costos']:
        plt.plot(resultado['costos'][:200], linewidth=2, label=f'α = {alpha}')

plt.title('Comparación Detallada (Primeras 200 iteraciones)')
plt.xlabel('Iteraciones')
plt.ylabel('Costo')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: Velocidad de convergencia
plt.subplot(2, 2, 4)
# Calcular iteraciones para alcanzar 95% de convergencia
iteraciones_conv = []
alphas_conv = []

for resultado in resultados_validos:
    costos = np.array(resultado['costos'])
    if len(costos) > 50:
        costo_inicial = costos[0]
        costo_final = costos[-1]
        umbral_95 = costo_inicial - 0.95 * (costo_inicial - costo_final)
        
        # Encontrar cuándo alcanza 95% de convergencia
        idx_conv = np.where(costos <= umbral_95)[0]
        if len(idx_conv) > 0:
            iteraciones_conv.append(idx_conv[0])
            alphas_conv.append(resultado['alpha'])

if iteraciones_conv:
    plt.scatter(alphas_conv, iteraciones_conv, s=100, alpha=0.7)
    plt.title('Velocidad de Convergencia\n(Iteraciones para 95% convergencia)')
    plt.xlabel('Learning Rate (α)')
    plt.ylabel('Iteraciones')
    plt.xscale('log')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Análisis de Learning Rate:")
print(f"   🟢 α muy pequeño (< 0.01): Converge lento")
print(f"   🟡 α moderado (0.01-0.3): Convergencia óptima")
print(f"   🔴 α muy grande (> 1.0): Puede divergir")
print(f"   🎯 Rango recomendado: 0.01 - 0.3 para datos normalizados")

## 5. 🚀 Modelo Optimizado Final

### 5.1 Entrenar el Mejor Modelo

In [ ]:
# Entrenar modelo final con configuración óptima
print(f"🚀 ENTRENAMIENTO FINAL: Modelo Optimizado")
print(f"=" * 50)

# Configuración óptima
alpha_optimo = mejor_resultado['alpha'] if 'mejor_resultado' in locals() else 0.1
iteraciones_final = 1000

print(f"Configuración:")
print(f"   • Feature scaling: Z-score normalization")
print(f"   • Learning rate: α = {alpha_optimo}")
print(f"   • Iteraciones: {iteraciones_final}")

# Entrenar modelo final
w_final, b_final, costos_final, params_final = gradient_descent_mejorado(
    X_norm, y_train, np.zeros(n), 0., alpha_optimo, iteraciones_final, verbose=True
)

print(f"\n🎉 RESULTADOS FINALES:")
print(f"   Costo inicial: {costos_final[0]:.4f}")
print(f"   Costo final:   {costos_final[-1]:.4f}")
print(f"   Reducción:     {((costos_final[0] - costos_final[-1])/costos_final[0]*100):.1f}%")

print(f"\n📊 Parámetros del modelo normalizado:")
for i, (feature, peso) in enumerate(zip(feature_names, w_final)):
    print(f"   w[{i}] ({feature}): {peso:8.4f}")
print(f"   b (bias): {b_final:8.4f}")

# Evaluar modelo
predicciones_norm = [np.dot(X_norm[i], w_final) + b_final for i in range(len(X_norm))]
residuos = [pred - real for pred, real in zip(predicciones_norm, y_train)]

# Métricas
mae = np.mean([abs(r) for r in residuos])
rmse = np.sqrt(np.mean([r**2 for r in residuos]))
ss_res = sum([r**2 for r in residuos])
ss_tot = sum([(real - np.mean(y_train))**2 for real in y_train])
r2 = 1 - (ss_res / ss_tot)

print(f"\n📈 Métricas de rendimiento:")
print(f"   MAE (Error Absoluto Medio):  ${mae:.1f}k")
print(f"   RMSE (Raíz Error Cuadrático): ${rmse:.1f}k")
print(f"   R² (Coef. Determinación):     {r2:.4f} ({r2*100:.1f}% varianza explicada)")

if r2 > 0.9:
    print(f"   ✅ Excelente ajuste (R² > 0.9)")
elif r2 > 0.8:
    print(f"   ✅ Muy buen ajuste (R² > 0.8)")
else:
    print(f"   ⚠️  Ajuste aceptable pero mejorable")

### 5.2 Función de Predicción para Nuevos Datos

In [ ]:
# Crear función de predicción completa que maneje normalización
def predecir_precio_casa(tamaño, habitaciones, pisos, edad, w, b, mu, sigma):
    """
    Predice el precio de una casa usando el modelo entrenado
    
    Args:
        tamaño: Tamaño en pies cuadrados
        habitaciones: Número de habitaciones
        pisos: Número de pisos
        edad: Edad en años
        w, b: Parámetros del modelo normalizado
        mu, sigma: Estadísticas de normalización
    
    Returns:
        precio: Precio predicho en miles de dólares
    """
    # 1. Crear vector de features
    x_casa = np.array([tamaño, habitaciones, pisos, edad])
    
    # 2. Normalizar usando estadísticas del entrenamiento
    x_norm = (x_casa - mu) / sigma
    
    # 3. Hacer predicción
    precio = np.dot(x_norm, w) + b
    
    return precio

# Función para análisis detallado
def analisis_prediccion_detallado(tamaño, habitaciones, pisos, edad, w, b, mu, sigma):
    """
    Análisis detallado de una predicción mostrando cada paso
    """
    print(f"🔍 ANÁLISIS DETALLADO DE PREDICCIÓN")
    print(f"=" * 40)
    print(f"Casa: {tamaño} sqft, {habitaciones} hab, {pisos} pisos, {edad} años")
    
    # Paso 1: Features originales
    x_original = np.array([tamaño, habitaciones, pisos, edad])
    print(f"\n1️⃣ Features originales: {x_original}")
    
    # Paso 2: Normalización
    x_norm = (x_original - mu) / sigma
    print(f"\n2️⃣ Normalización (z-score):")
    for i, (feature, orig, norm, m, s) in enumerate(zip(feature_names, x_original, x_norm, mu, sigma)):
        print(f"   {feature}: ({orig:.0f} - {m:.1f}) / {s:.1f} = {norm:.3f}")
    
    # Paso 3: Cálculo de predicción
    print(f"\n3️⃣ Cálculo de predicción:")
    contribuciones = w * x_norm
    for i, (feature, peso, x_n, contrib) in enumerate(zip(feature_names, w, x_norm, contribuciones)):
        print(f"   {feature}: {peso:.4f} × {x_n:.3f} = {contrib:.3f}")
    
    suma_contrib = np.sum(contribuciones)
    precio_final = suma_contrib + b
    
    print(f"   Suma de contribuciones: {suma_contrib:.3f}")
    print(f"   + Bias: {b:.3f}")
    print(f"   = Precio final: ${precio_final:.0f}k")
    
    return precio_final

# Ejemplos de predicción
ejemplos_casas = [
    (1200, 3, 1, 15, "Casa familiar típica"),
    (2500, 4, 2, 5, "Casa grande y nueva"),
    (800, 2, 1, 40, "Casa pequeña, antigua"),
    (3200, 5, 2, 2, "Mansión nueva"),
]

print(f"\n🏠 PREDICCIONES PARA CASAS NUEVAS")
print(f"=" * 45)
print(f"{'Descripción':<20} {'Features':<20} {'Precio':<12}")
print("-" * 55)

for tamaño, hab, pisos, edad, desc in ejemplos_casas:
    precio = predecir_precio_casa(tamaño, hab, pisos, edad, w_final, b_final, X_mu, X_sigma)
    features_str = f"{tamaño}sq,{hab}b,{pisos}f,{edad}y"
    print(f"{desc:<20} {features_str:<20} ${precio:<11.0f}")

# Análisis detallado de un ejemplo
print(f"\n" + "="*50)
ejemplo_detalle = ejemplos_casas[0]  # Casa familiar típica
analisis_prediccion_detallado(ejemplo_detalle[0], ejemplo_detalle[1], 
                             ejemplo_detalle[2], ejemplo_detalle[3], 
                             w_final, b_final, X_mu, X_sigma)

### 5.3 Comparación Final: Antes vs Después

In [ ]:
# Comparación final completa
plt.figure(figsize=(16, 10))

# Datos para comparación (recrear el entrenamiento sin normalización para comparar)
w_sin_norm, b_sin_norm, costos_sin_norm, _ = gradient_descent_mejorado(
    X_train, y_train, np.zeros(n), 0., 5e-7, 1000, verbose=False
)

# Subplot 1: Comparación de convergencia
plt.subplot(2, 3, 1)
plt.plot(costos_sin_norm, 'r-', linewidth=2, label='Sin normalización', alpha=0.8)
plt.plot(costos_final, 'g-', linewidth=2, label='Con normalización optimizada', alpha=0.8)
plt.title('Comparación de Convergencia')
plt.xlabel('Iteraciones')
plt.ylabel('Costo')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Subplot 2: Predicciones vs valores reales (modelo optimizado)
plt.subplot(2, 3, 2)
plt.scatter(predicciones_norm, y_train, alpha=0.7, s=60, color='green')
min_val = min(min(predicciones_norm), min(y_train))
max_val = max(max(predicciones_norm), max(y_train))
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfecto')
plt.title('Predicciones vs Reales (Optimizado)')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Distribución de residuos
plt.subplot(2, 3, 3)
plt.hist(residuos, bins=10, alpha=0.7, edgecolor='black', color='green')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribución de Residuos')
plt.xlabel('Residuos ($k)')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

# Subplot 4: Importancia de features (pesos normalizados)
plt.subplot(2, 3, 4)
importancia = np.abs(w_final)
indices_ordenados = np.argsort(importancia)[::-1]
colores_bars = ['gold', 'silver', 'chocolate', 'lightblue']

bars = plt.bar(range(n), importancia[indices_ordenados], 
               color=[colores_bars[i] for i in indices_ordenados], 
               alpha=0.7, edgecolor='black')
plt.title('Importancia de Features\n(Magnitud de Pesos Normalizados)')
plt.xlabel('Features')
plt.ylabel('|Peso|')
plt.xticks(range(n), [feature_names[i].split()[0] for i in indices_ordenados], rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for bar, valor in zip(bars, importancia[indices_ordenados]):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
             f'{valor:.3f}', ha='center', va='bottom')

# Subplot 5: Comparación de métricas
plt.subplot(2, 3, 5)
# Calcular métricas del modelo sin normalizar
pred_sin_norm = [np.dot(X_train[i], w_sin_norm) + b_sin_norm for i in range(len(X_train))]
residuos_sin_norm = [pred - real for pred, real in zip(pred_sin_norm, y_train)]
mae_sin_norm = np.mean([abs(r) for r in residuos_sin_norm])
rmse_sin_norm = np.sqrt(np.mean([r**2 for r in residuos_sin_norm]))

metricas_nombres = ['MAE ($k)', 'RMSE ($k)']
sin_norm_vals = [mae_sin_norm, rmse_sin_norm]
con_norm_vals = [mae, rmse]

x_pos = np.arange(len(metricas_nombres))
width = 0.35

plt.bar(x_pos - width/2, sin_norm_vals, width, label='Sin normalización', 
        color='red', alpha=0.7, edgecolor='black')
plt.bar(x_pos + width/2, con_norm_vals, width, label='Con normalización', 
        color='green', alpha=0.7, edgecolor='black')

plt.title('Comparación de Métricas')
plt.xlabel('Métrica')
plt.ylabel('Valor')
plt.xticks(x_pos, metricas_nombres)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Subplot 6: Resumen de mejoras
plt.subplot(2, 3, 6)
plt.text(0.1, 0.9, '🎯 MEJORAS LOGRADAS', fontsize=14, fontweight='bold', 
         transform=plt.gca().transAxes)

mejoras_texto = f"""
⚡ Learning Rate:
   • Antes: 5.0e-7
   • Después: {alpha_optimo}
   • Mejora: {alpha_optimo/5e-7:.0f}x más rápido

📊 Convergencia:
   • Más estable y predecible
   • Sin oscilaciones
   • Gradientes balanceados

🎯 Rendimiento:
   • MAE: {mae_sin_norm:.1f} → {mae:.1f}
   • RMSE: {rmse_sin_norm:.1f} → {rmse:.1f}
   • R²: {r2:.3f} ({r2*100:.1f}%)

✅ Resultado:
   Modelo robusto y eficiente
   para producción
"""

plt.text(0.05, 0.8, mejoras_texto, fontsize=10, 
         transform=plt.gca().transAxes, verticalalignment='top')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"\n🎉 RESUMEN FINAL")
print(f"=" * 30)
print(f"✅ Feature scaling implementado exitosamente")
print(f"✅ Learning rate optimizado: {alpha_optimo/5e-7:.0f}x más rápido")
print(f"✅ Modelo con R² = {r2:.3f} ({r2*100:.1f}% de varianza explicada)")
print(f"✅ Convergencia estable y predecible")
print(f"✅ Listo para hacer predicciones en nuevos datos")

## 6. 🧪 Ejercicios Prácticos

### Ejercicio 1: Implementar Diferentes Técnicas de Normalización

In [ ]:
# Ejercicio 1: Comparar técnicas de normalización
print(f"🧪 EJERCICIO 1: Comparación de Técnicas de Normalización")
print(f"=" * 60)

def robust_normalize_features(X):
    """
    Normalización robusta usando mediana y rango intercuartil
    Menos sensible a outliers que Z-score
    """
    median = np.median(X, axis=0)
    q75 = np.percentile(X, 75, axis=0)
    q25 = np.percentile(X, 25, axis=0)
    iqr = q75 - q25
    
    # Evitar división por cero
    iqr[iqr == 0] = 1
    
    X_robust = (X - median) / iqr
    return X_robust, median, iqr

def unit_vector_normalize(X):
    """
    Normalización de vector unitario (L2 normalization)
    Cada ejemplo tiene norma = 1
    """
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Evitar división por cero
    return X / norms

# Aplicar todas las técnicas
tecnicas = {
    'Original': (X_train, None, None),
    'Z-Score': zscore_normalize_features(X_train),
    'Min-Max': minmax_normalize_features(X_train),
    'Robust': robust_normalize_features(X_train),
    'Unit Vector': (unit_vector_normalize(X_train), None, None)
}

# Entrenar modelo con cada técnica
resultados_tecnicas = {}

for nombre, (X_norm_tech, _, _) in tecnicas.items():
    if nombre == 'Original':
        alpha_tech = 5e-7  # Learning rate pequeño para datos originales
    else:
        alpha_tech = 0.1   # Learning rate grande para datos normalizados
    
    try:
        w_tech, b_tech, costos_tech, _ = gradient_descent_mejorado(
            X_norm_tech, y_train, np.zeros(n), 0., alpha_tech, 500, verbose=False
        )
        
        # Métricas
        pred_tech = [np.dot(X_norm_tech[i], w_tech) + b_tech for i in range(len(X_norm_tech))]
        mae_tech = np.mean([abs(pred - real) for pred, real in zip(pred_tech, y_train)])
        
        resultados_tecnicas[nombre] = {
            'costo_final': costos_tech[-1],
            'mae': mae_tech,
            'convergencia': len([c for c in costos_tech if not np.isnan(c)]),
            'alpha': alpha_tech
        }
    except:
        resultados_tecnicas[nombre] = {
            'costo_final': float('inf'),
            'mae': float('inf'),
            'convergencia': 0,
            'alpha': alpha_tech
        }

# Mostrar resultados
print(f"{'Técnica':<15} {'α':<10} {'Costo Final':<12} {'MAE':<8} {'Convergencia':<12}")
print("-" * 60)

for nombre, resultado in resultados_tecnicas.items():
    costo_str = f"{resultado['costo_final']:.2f}" if resultado['costo_final'] != float('inf') else "Falló"
    mae_str = f"{resultado['mae']:.1f}" if resultado['mae'] != float('inf') else "N/A"
    
    print(f"{nombre:<15} {resultado['alpha']:<10.0e} {costo_str:<12} {mae_str:<8} {'✅' if resultado['convergencia'] > 400 else '❌':<12}")

# Encontrar la mejor técnica
tecnicas_validas = {k: v for k, v in resultados_tecnicas.items() if v['mae'] != float('inf')}
if tecnicas_validas:
    mejor_tecnica = min(tecnicas_validas, key=lambda x: tecnicas_validas[x]['mae'])
    print(f"\n🏆 Mejor técnica: {mejor_tecnica}")
    print(f"   MAE: {tecnicas_validas[mejor_tecnica]['mae']:.1f}")
    print(f"   Permite α = {tecnicas_validas[mejor_tecnica]['alpha']:.0e}")

print(f"\n💡 Conclusiones:")
print(f"   • Z-Score y Min-Max suelen ser las mejores opciones")
print(f"   • Robust normalization es útil con outliers")
print(f"   • Unit vector es específica para ciertos casos")
print(f"   • Normalización permite learning rates 10,000x más grandes")

### Ejercicio 2: Análisis de Sensibilidad del Learning Rate

In [ ]:
# Ejercicio 2: Búsqueda sistemática de learning rate óptimo
print(f"🧪 EJERCICIO 2: Búsqueda Sistemática de Learning Rate")
print(f"=" * 55)

def busqueda_learning_rate_detallada(X, y, alpha_min=1e-4, alpha_max=10, num_puntos=20):
    """
    Búsqueda sistemática de learning rate con análisis detallado
    """
    # Generar rango logarítmico de learning rates
    alphas = np.logspace(np.log10(alpha_min), np.log10(alpha_max), num_puntos)
    
    resultados_detallados = []
    
    for alpha in alphas:
        try:
            # Entrenamiento corto para evaluación rápida
            w, b, costos, _ = gradient_descent_mejorado(
                X, y, np.zeros(X.shape[1]), 0., alpha, 200, verbose=False
            )
            
            # Análisis de convergencia
            if len(costos) < 50:
                status = "Error temprano"
                velocidad_conv = 0
                estabilidad = 0
            elif np.isnan(costos[-1]) or np.isinf(costos[-1]):
                status = "Divergencia (NaN/Inf)"
                velocidad_conv = 0
                estabilidad = 0
            elif costos[-1] > costos[0] * 2:
                status = "Divergencia (Costo aumenta)"
                velocidad_conv = 0
                estabilidad = 0
            else:
                status = "Convergencia"
                # Velocidad: qué tan rápido baja el costo en las primeras iteraciones
                if len(costos) >= 50:
                    velocidad_conv = (costos[0] - costos[49]) / costos[0]
                else:
                    velocidad_conv = (costos[0] - costos[-1]) / costos[0]
                
                # Estabilidad: variación en las últimas iteraciones
                ultimos_costos = costos[-20:] if len(costos) >= 20 else costos
                estabilidad = 1 / (1 + np.std(ultimos_costos))
            
            resultados_detallados.append({
                'alpha': alpha,
                'status': status,
                'costo_final': costos[-1] if costos and not np.isnan(costos[-1]) and not np.isinf(costos[-1]) else float('inf'),
                'velocidad_convergencia': velocidad_conv,
                'estabilidad': estabilidad,
                'score_total': velocidad_conv * estabilidad  # Métrica combinada
            })
            
        except Exception as e:
            resultados_detallados.append({
                'alpha': alpha,
                'status': f"Error: {str(e)[:20]}",
                'costo_final': float('inf'),
                'velocidad_convergencia': 0,
                'estabilidad': 0,
                'score_total': 0
            })
    
    return resultados_detallados

# Realizar búsqueda detallada
print("Realizando búsqueda sistemática...")
resultados_lr_detallados = busqueda_learning_rate_detallada(X_norm, y_train)

# Mostrar resultados tabulados
print(f"\n{'α':<10} {'Status':<15} {'Costo Final':<12} {'Velocidad':<10} {'Estabilidad':<12} {'Score':<8}")
print("-" * 75)

for resultado in resultados_lr_detallados[:10]:  # Mostrar primeros 10
    costo_str = f"{resultado['costo_final']:.2f}" if resultado['costo_final'] != float('inf') else "∞"
    print(f"{resultado['alpha']:<10.1e} {resultado['status']:<15} {costo_str:<12} "
          f"{resultado['velocidad_convergencia']:<10.3f} {resultado['estabilidad']:<12.3f} "
          f"{resultado['score_total']:<8.3f}")

# Encontrar el mejor según score combinado
resultados_validos_det = [r for r in resultados_lr_detallados if r['status'] == 'Convergencia']
if resultados_validos_det:
    mejor_lr_detallado = max(resultados_validos_det, key=lambda x: x['score_total'])
    print(f"\n🎯 Learning rate óptimo (análisis detallado):")
    print(f"   α = {mejor_lr_detallado['alpha']:.3f}")
    print(f"   Score total: {mejor_lr_detallado['score_total']:.3f}")
    print(f"   Velocidad de convergencia: {mejor_lr_detallado['velocidad_convergencia']:.3f}")
    print(f"   Estabilidad: {mejor_lr_detallado['estabilidad']:.3f}")

# Visualización de la búsqueda
plt.figure(figsize=(12, 4))

alphas_plot = [r['alpha'] for r in resultados_lr_detallados]
scores_plot = [r['score_total'] for r in resultados_lr_detallados]
colores_status = ['green' if r['status'] == 'Convergencia' else 'red' for r in resultados_lr_detallados]

plt.subplot(1, 2, 1)
plt.scatter(alphas_plot, scores_plot, c=colores_status, alpha=0.7, s=50)
plt.xscale('log')
plt.title('Score de Learning Rate vs α')
plt.xlabel('Learning Rate (α)')
plt.ylabel('Score (Velocidad × Estabilidad)')
plt.grid(True, alpha=0.3)

# Marcar el mejor
if resultados_validos_det:
    plt.scatter([mejor_lr_detallado['alpha']], [mejor_lr_detallado['score_total']], 
               c='blue', s=200, marker='*', edgecolor='black', linewidth=2,
               label=f"Óptimo: α={mejor_lr_detallado['alpha']:.3f}")
    plt.legend()

# Heatmap de regiones
plt.subplot(1, 2, 2)
velocidades = [r['velocidad_convergencia'] for r in resultados_lr_detallados]
estabilidades = [r['estabilidad'] for r in resultados_lr_detallados]

plt.scatter(velocidades, estabilidades, c=scores_plot, s=100, alpha=0.7, cmap='viridis')
plt.colorbar(label='Score Total')
plt.title('Velocidad vs Estabilidad')
plt.xlabel('Velocidad de Convergencia')
plt.ylabel('Estabilidad')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Conclusiones del análisis sistemático:")
print(f"   • Zona óptima: α entre 0.03 y 0.3")
print(f"   • Balance ideal entre velocidad y estabilidad")
print(f"   • Evitar α < 0.001 (muy lento) y α > 1.0 (inestable)")

## 📚 Resumen y Conceptos Clave

### ✅ Lo que has aprendido:

#### 1. **Problem Diagnosis**:
   - **Features con escalas muy diferentes** causan convergencia lenta
   - **Gradientes desbalanceados** requieren learning rates muy pequeños
   - **Superficies de costo elípticas** vs **circulares**

#### 2. **Feature Scaling Techniques**:
   - **Z-Score**: $(x - \mu) / \sigma$ → Media=0, Std=1
   - **Min-Max**: $(x - x_{min}) / (x_{max} - x_{min})$ → Rango [0,1]
   - **Robust**: $(x - median) / IQR$ → Resistente a outliers
   - **Unit Vector**: $x / ||x||$ → Norma = 1

#### 3. **Impact on Gradient Descent**:
   - **Learning rate**: 10,000x más grande con normalización
   - **Convergencia**: Más rápida, estable y predecible
   - **Gradientes**: Magnitudes similares entre features

#### 4. **Learning Rate Optimization**:
   - **Systematic search**: Explorar rango logarítmico
   - **Balance**: Velocidad vs estabilidad
   - **Zona óptima**: 0.01 - 0.3 para datos normalizados

#### 5. **Production Ready Model**:
   - **Normalización**: Aplicar a nuevos datos usando estadísticas del entrenamiento
   - **Pipeline completo**: Normalizar → Predecir → Desnormalizar (si necesario)
   - **Métricas**: R² = {r2:.3f} ({r2*100:.1f}% varianza explicada)

### 🚀 Mejoras Logradas:
- **Velocidad**: {alpha_optimo/5e-7:.0f}x más rápido
- **Estabilidad**: Sin oscilaciones ni divergencia
- **Rendimiento**: MAE reducido, R² mejorado
- **Robustez**: Modelo listo para producción

### 🎯 Próximos pasos:
- **Feature Engineering**: Crear features polinomiales y transformaciones
- **Regularización**: L1/L2 para prevenir overfitting
- **Advanced Optimization**: Adam, RMSprop, momentum
- **Cross-validation**: Validación robusta del modelo

### 💡 Best Practices:
1. **Siempre normalizar** cuando features tienen escalas diferentes
2. **Guardar estadísticas** (μ, σ) del conjunto de entrenamiento
3. **Aplicar misma normalización** a datos nuevos
4. **Experimentar con learning rates** en rango logarítmico
5. **Monitorear convergencia** con múltiples métricas
6. **Validar en datos de prueba** antes de producción

### ⚠️ Cuidados importantes:
- **Nunca normalizar** usando estadísticas de datos de prueba
- **Feature scaling no siempre mejora** todos los algoritmos
- **Learning rate muy alto** puede causar divergencia
- **Interpretar pesos** de modelo normalizado requiere cuidado

**Este notebook te ha dado las herramientas fundamentales para optimizar gradient descent y crear modelos de ML robustos y eficientes.**